In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [6]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

In [19]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

# Mens

In [20]:
category_id={'cat100274':'Kurtas',
            'cat100279':'Nehru-jackets-blazers',
            'cat100280':'Pyjamas and Churidars',
            'cat100283':'Salwars and Dhotis',
            'cat100287':'Shirts',
            'cat100290':'Shawls and Mufflers',
            'cat100294':'Trousers and Shorts',
            'cat100271':"Bags & Accessories",
            'cat100272':'Footwear'}

## Clothings

In [21]:
mens_links=[]

for  i in tqdm(range(0,3078,55)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100273&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            mens=[]
            mens.append('https://www.fabindia.com/'+j['route'])
            mens.append(j['id'])
            mens_links.append(mens)
    except:
        pass

100%|██████████| 56/56 [01:41<00:00,  1.82s/it]


## Footwear

In [22]:
for  i in tqdm(range(0,110,55)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100272&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            mens=[]
            mens.append('https://www.fabindia.com/'+j['route'])
            mens.append(j['id'])
            mens_links.append(mens)
    except:
        pass

100%|██████████| 2/2 [00:02<00:00,  1.44s/it]


## Bags

In [23]:
for  i in tqdm(range(0,1,55)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100271&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            mens=[]
            mens.append('https://www.fabindia.com/'+j['route'])
            mens.append(j['id'])
            mens_links.append(mens)
    except:
        pass

100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


In [24]:
err=[]
for i in tqdm(mens_links):
    try:
        result = requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?productIds={i[1]}',headers=headers)
        get = result.content
        js=json.loads(get)
        
        product_name=js['items'][0]['primaryImageAltText']
        product_brand=js['items'][0]['brand']
        product_link=i[0]
        data=js['items'][0]['parentCategoryIdPath'].split('>')
        for i in data:
            if category_id.get(i) !=None:
                cat=category_id.get(i)
                break
        product_cat=cat
        product_des=js['items'][0]['longDescription']
        product_mrp=int(js['items'][0]['listPrices']['INR'])
        if js['items'][0]['salePrices']['INR'] == None:
            product_price=product_mrp
        else:
            product_price=int(js['items'][0]['salePrices']['INR'])
            
        images=[]
        for j in js['items'][0]['thumbImageURLs']:
            images.append("https://www.fabindia.com/"+j.split('&')[0])
        
        sizes=[]
        for k in js['items'][0]['childSKUs']:
            sizes.append(k['x_actualSize'])
        size_avl=','.join(sizes)
        primary_link=images[0]
        secondary_link=','.join(images[1:])

    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)


100%|██████████| 3179/3179 [38:00<00:00,  1.39it/s]


In [25]:
mens=pd.DataFrame({
    
    "Website":'https://www.fabindia.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [26]:
mens.shape

(3179, 13)

In [28]:
mens.to_csv("fab_mens.csv",index=False)

In [29]:
con=sqlite3.connect("fabindia.db")  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(Product_Link)):
    try:
        cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.craftsvilla.com/',Product_Link[i],Product_Name[i],Product_Brand[i],Cat_list[i],Size_Available[i],Price[i],MRP[i],Gender[i],Description[i],Primary_Image_Link[i],Secondary_Image_Link[i],''))
    except:
        pass
con.commit()
con.close()

# Women

In [30]:
womens_cat_id={'cat100315':'capes-shrugs',
               'cat100316':'dresses',
               'cat100318':'dupattas',
               'cat100320':'Jackets & Coats',
               'cat100323':'Kurtas',
               'cat100328':'Pants & Palazzos',
               'cat100332':'Salwars, Churidars & Patialas',
               'cat100336':'Saris & Blouses',
               'cat100339':'awls & Mufflers',
               'cat100342':'Skirts',
               'cat320004':"Stoles and Scarves",
               'cat100346':"Tops & Shirts",
               'cat100349':"Tunics",
               'cat100300':'Bags',
               'cat100303':'Jewellery',
               'cat100302':'Footwear'
               }

## Clothings

In [38]:
womens_links=[]

for  i in tqdm(range(0,8580,54)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100314&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            womens=[]
            womens.append('https://www.fabindia.com/'+j['route'])
            womens.append(j['id'])
            womens_links.append(womens)
    except:
        pass

100%|██████████| 159/159 [01:39<00:00,  1.60it/s]


## Bags

In [32]:
womens_links=[]
for  i in tqdm(range(0,880,55)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100300&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            womens=[]
            womens.append('https://www.fabindia.com/'+j['route'])
            womens.append(j['id'])
            womens_links.append(womens)
    except:
        pass

100%|██████████| 16/16 [00:26<00:00,  1.67s/it]


## Footwear

In [33]:
for  i in tqdm(range(0,220,55)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100302&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            womens=[]
            womens.append('https://www.fabindia.com/'+j['route'])
            womens.append(j['id'])
            womens_links.append(womens)
    except:
        pass

100%|██████████| 4/4 [00:08<00:00,  2.09s/it]


## Jewelley

In [34]:
for  i in tqdm(range(0,4400,55)):
    try:
        result=requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?totalResults=true&totalExpandedResults=true&catalogId=cloudCatalog&offset={i}&limit=55&sort=creationDate%3Adesc&categoryId=cat100302&includeChildren=true&storePriceListGroupId=INR',headers=headers)
        get=result.content
        js=json.loads(get)
        for j in js['items']:
            womens=[]
            womens.append('https://www.fabindia.com/'+j['route'])
            womens.append(j['id'])
            womens_links.append(womens)
    except:
        pass

100%|██████████| 80/80 [01:08<00:00,  1.16it/s]


In [39]:
err=[]
for i in tqdm(womens_links):
    try:
        result = requests.get(f'https://www.fabindia.com/ccstoreui/v1/products?productIds={i[1]}',headers=headers)
        get = result.content
        js=json.loads(get)
        
        product_name=js['items'][0]['primaryImageAltText']
        product_brand=js['items'][0]['brand']
        product_link=i[0]
        data=js['items'][0]['parentCategoryIdPath'].split('>')
        for i in data:
            if womens_cat_id.get(i) !=None:
                cat=womens_cat_id.get(i)
                break
        product_cat=cat
        product_des=js['items'][0]['longDescription']
        product_mrp=int(js['items'][0]['listPrices']['INR'])
        if js['items'][0]['salePrices']['INR'] == None:
            product_price=product_mrp
        else:
            product_price=int(js['items'][0]['salePrices']['INR'])
            
        images=[]
        for j in js['items'][0]['thumbImageURLs']:
            images.append("https://www.fabindia.com/"+j.split('&')[0])
        
        sizes=[]
        for k in js['items'][0]['childSKUs']:
            sizes.append(k['x_actualSize'])
        size_avl=','.join(sizes)
        primary_link=images[0]
        secondary_link=','.join(images[1:])

    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)


100%|██████████| 8745/8745 [1:52:44<00:00,  1.29it/s]   


In [40]:
out=pd.DataFrame({
    
    "Website":'https://www.fabindia.com/',
    "Product_Link":Product_Link,
    "Product_Name":Product_Name,
    "Product_Brand":Product_Brand,
    "Product_Category":Cat_list,
    "Size_Avail":Size_Available,
    "Price":Price,
    "MRP":MRP,
    "Gender":Gender,
    "Description":Description,
    "Primary_Image_Links":Primary_Image_Link,
    "Secondary_Image_Links":Secondary_Image_Link,
    "Affiliate_Link":''
    
})

In [41]:
out.shape

(13147, 13)

In [43]:
out.to_csv("fabindia.csv",index=False)

In [50]:
con=sqlite3.connect("fabindia.db")  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
out.to_sql('product_details',con,if_exists='replace', index = False)
con.commit()
con.close()

In [1]:
li=[1,2,3,4]

In [4]:
li[-2]

3